In [ ]:
from google.colab import drive
try:
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
    print("Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting drive: {e}")
    raise

Mounting Google Drive...
Mounted at /content/drive
Drive mounted successfully!


In [ ]:
import os
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import pandas as pd
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

try:
    print("Initializing CLIP model and processor...")
    # Setup CLIP model and processor
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
except Exception as e:
    print(f"Error initializing CLIP: {e}")
    raise

# Define file paths
try:
    image_folders = {
        'Flux_Single': '/content/drive/My Drive/final/Flux_Single_750',
        'Flux_Multi_V2': '/content/drive/My Drive/final/Flux_Multi_V2_750',
        'Alt_Single': '/content/drive/My Drive/final/Alt_Single_3750',
        'Alt_Multi_V2': '/content/drive/My Drive/final/Alt_Multi_V2_3750'
    }

    # Verify folders exist
    for name, path in image_folders.items():
        if not os.path.exists(path):
            print(f"Warning: Folder not found - {path}")
except Exception as e:
    print(f"Error setting up file paths: {e}")
    raise

def get_unique_landmarks():
    """Extract unique landmarks from all Excel files"""
    try:
        all_landmarks = set()
        files = ['Flux_Single.xlsx', 'Flux_Multi_V2.xlsx', 'Alt_Single.xlsx', 'Alt_Multi_V2.xlsx']

        for file in files:
            try:
                print(f"\nReading landmarks from {file}")
                df = pd.read_excel(file)
                if 'Landmark' in df.columns:
                    landmarks = df['Landmark'].unique()
                    all_landmarks.update(landmarks)
                    print(f"Found {len(landmarks)} unique landmarks in {file}")
            except Exception as e:
                print(f"Error reading landmarks from {file}: {e}")
                continue

        all_landmarks = list(all_landmarks)
        print(f"\nTotal unique landmarks across all files: {len(all_landmarks)}")
        print("List of all unique landmarks:")
        for idx, landmark in enumerate(sorted(all_landmarks), 1):
            print(f"{idx}. {landmark}")
        return all_landmarks
    except Exception as e:
        print(f"Error in get_unique_landmarks: {e}")
        raise

def compute_clip_score(image_path, text_prompt):
    """Compute CLIP similarity score between image and text"""
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(
            text=[text_prompt],
            images=image,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            similarity = F.cosine_similarity(outputs.image_embeds, outputs.text_embeds).item()
        return similarity
    except Exception as e:
        print(f"Error computing CLIP score: {e}")
        print(f"Image path: {image_path}")
        print(f"Text prompt: {text_prompt}")
        return None

def get_random_landmark(current_landmark, all_landmarks):
    """Get a random landmark different from the current one"""
    try:
        possible_landmarks = [l for l in all_landmarks if l != current_landmark]
        if not possible_landmarks:
            raise ValueError(f"No alternative landmarks available for {current_landmark}")
        return random.choice(possible_landmarks)
    except Exception as e:
        print(f"Error in get_random_landmark: {e}")
        return None

def process_flux_file(excel_path, image_folder, all_landmarks):
    """Process Flux files (Single and Multi)"""
    try:
        print(f"\nProcessing Flux file: {excel_path}")
        df = pd.read_excel(excel_path)
        df['New_Knowledge_Score'] = None

        for idx, row in tqdm(df.iterrows(), total=len(df)):
            try:
                # Get image path
                image_path = os.path.join(image_folder, row['Image'])
                if not os.path.exists(image_path):
                    image_path = os.path.join(image_folder, row['Image'] + '.png')
                    if not os.path.exists(image_path):
                        print(f"\nError: Image not found - {row['Image']}")
                        continue

                # Get random landmark and create new prompt
                current_landmark = row['Landmark']
                new_landmark = get_random_landmark(current_landmark, all_landmarks)
                if new_landmark is None:
                    continue

                new_prompt = row['Prompt'].replace(current_landmark, new_landmark)
                print(f"\nRow {idx}:")
                print(f"Original landmark: {current_landmark}")
                print(f"Randomly replaced with: {new_landmark}")

                # Compute new score
                new_score = compute_clip_score(image_path, new_prompt)
                df.at[idx, 'New_Knowledge_Score'] = new_score

                # Save periodically
                if idx % 10 == 0:
                    df.to_excel(excel_path, index=False)
                    print(f"Progress saved at row {idx}")

            except Exception as e:
                print(f"Error processing row {idx} in {excel_path}: {e}")
                continue

        # Final save
        df.to_excel(excel_path, index=False)
        print(f"Completed processing {excel_path}")

    except Exception as e:
        print(f"Error processing file {excel_path}: {e}")

def process_alt_file(excel_path, image_folder, all_landmarks):
    """Process Alt files (Single and Multi) in batches of 5"""
    try:
        print(f"\nProcessing Alt file: {excel_path}")
        df = pd.read_excel(excel_path)
        df['New_Knowledge_Score'] = None

        # Process in batches of 5
        for start_idx in tqdm(range(0, len(df), 5)):
            try:
                batch = df.iloc[start_idx:start_idx + 5]
                print(f"\nProcessing batch starting at index {start_idx}")

                for idx, row in batch.iterrows():
                    try:
                        # Get image path
                        image_path = os.path.join(image_folder, row['Image'])
                        if not os.path.exists(image_path):
                            image_path = os.path.join(image_folder, row['Image'] + '.png')
                            if not os.path.exists(image_path):
                                print(f"\nError: Image not found - {row['Image']}")
                                continue

                        # Get random landmark and create new prompt
                        current_landmark = batch.iloc[0]['Landmark']  # Use landmark from English prompt
                        new_landmark = get_random_landmark(current_landmark, all_landmarks)
                        if new_landmark is None:
                            continue

                        new_prompt = batch.iloc[0]['Prompt'].replace(current_landmark, new_landmark)
                        print(f"\nBatch {start_idx//5}, Row {idx}:")
                        print(f"Original landmark: {current_landmark}")
                        print(f"Randomly replaced with: {new_landmark}")

                        # Compute new score
                        new_score = compute_clip_score(image_path, new_prompt)
                        df.at[idx, 'New_Knowledge_Score'] = new_score

                    except Exception as e:
                        print(f"Error processing row {idx} in batch {start_idx//5}: {e}")
                        continue

                # Save after each batch
                df.to_excel(excel_path, index=False)
                print(f"Batch {start_idx//5} completed and saved")

            except Exception as e:
                print(f"Error processing batch starting at {start_idx}: {e}")
                continue

        print(f"Completed processing {excel_path}")

    except Exception as e:
        print(f"Error processing file {excel_path}: {e}")

def main():
    try:
        # Set random seed for reproducibility
        random.seed(42)

        # Get unique landmarks
        all_landmarks = get_unique_landmarks()

        # Process Flux files
        print("\nProcessing Flux files...")
        process_flux_file('Flux_Single.xlsx', image_folders['Flux_Single'], all_landmarks)
        process_flux_file('Flux_Multi_V2.xlsx', image_folders['Flux_Multi_V2'], all_landmarks)

        # Process Alt files
        print("\nProcessing Alt files...")
        process_alt_file('Alt_Single.xlsx', image_folders['Alt_Single'], all_landmarks)
        process_alt_file('Alt_Multi_V2.xlsx', image_folders['Alt_Multi_V2'], all_landmarks)

        print("\nAll processing completed!")

    except Exception as e:
        print(f"Critical error in main: {e}")

if __name__ == "__main__":
    main()

Initializing CLIP model and processor...
Using device: cuda


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

Batch 747 completed and saved

Processing batch starting at index 3740

Batch 748, Row 3740:
Original landmark: Roman Theater of Cartagena
Randomly replaced with: Guggenheim Museum

Batch 748, Row 3741:
Original landmark: Roman Theater of Cartagena
Randomly replaced with: Royal Palace of Madrid

Batch 748, Row 3742:
Original landmark: Roman Theater of Cartagena
Randomly replaced with: Holocaust Memorial

Batch 748, Row 3743:
Original landmark: Roman Theater of Cartagena
Randomly replaced with: Meridian Gate of Huế

Batch 748, Row 3744:
Original landmark: Roman Theater of Cartagena
Randomly replaced with: Lincoln Memorial


100%|█████████▉| 749/750 [1:27:56<00:06,  6.39s/it]

Batch 748 completed and saved

Processing batch starting at index 3745

Batch 749, Row 3745:
Original landmark: Royal Palace of Madrid
Randomly replaced with: Sagrada Familia

Batch 749, Row 3746:
Original landmark: Royal Palace of Madrid
Randomly replaced with: Ho Chi Minh Mausoleum

Batch 749, Row 3747:
Original landmark: Royal Palace of Madrid
Randomly replaced with: Golden Gate Bridge

Batch 749, Row 3748:
Original landmark: Royal Palace of Madrid
Randomly replaced with: Thien Mu Pagoda

Batch 749, Row 3749:
Original landmark: Royal Palace of Madrid
Randomly replaced with: Ho Chi Minh Mausoleum


100%|██████████| 750/750 [1:28:03<00:00,  7.04s/it]

Batch 749 completed and saved
Completed processing Alt_Multi_V2.xlsx

All processing completed!


In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Files to process
files = ['Flux_Single.xlsx', 'Flux_Multi_V2.xlsx', 'Alt_Single.xlsx', 'Alt_Multi_V2.xlsx']

try:
    for file in files:
        print(f"\nProcessing {file}...")

        # Read Excel file
        df = pd.read_excel(file)

        # Calculate signed difference (original - new) and update New_Knowledge_Score column
        print(f"Calculating differences for {file}")
        df['New_Knowledge_Score'] = df['knowledge_score'] - df['New_Knowledge_Score']

        # Print some statistics
        print(f"Statistics for {file}:")
        print(f"Mean difference: {df['New_Knowledge_Score'].mean():.4f}")
        print(f"Max positive difference: {df['New_Knowledge_Score'].max():.4f}")
        print(f"Max negative difference: {df['New_Knowledge_Score'].min():.4f}")

        # Save back to same file
        df.to_excel(file, index=False)
        print(f"Updated {file} with differences")

    print("\nAll files processed successfully!")

except Exception as e:
    print(f"Error occurred: {e}")


Processing Flux_Single.xlsx...
Calculating differences for Flux_Single.xlsx
Statistics for Flux_Single.xlsx:
Mean difference: 0.0561
Max positive difference: 0.2117
Max negative difference: -0.0592
Updated Flux_Single.xlsx with differences

Processing Flux_Multi_V2.xlsx...
Calculating differences for Flux_Multi_V2.xlsx
Statistics for Flux_Multi_V2.xlsx:
Mean difference: 0.0365
Max positive difference: 0.1546
Max negative difference: -0.0588
Updated Flux_Multi_V2.xlsx with differences

Processing Alt_Single.xlsx...
Calculating differences for Alt_Single.xlsx
Statistics for Alt_Single.xlsx:
Mean difference: 0.0680
Max positive difference: 0.2249
Max negative difference: -0.0867
Updated Alt_Single.xlsx with differences

Processing Alt_Multi_V2.xlsx...
Calculating differences for Alt_Multi_V2.xlsx
Statistics for Alt_Multi_V2.xlsx:
Mean difference: 0.0373
Max positive difference: 0.1890
Max negative difference: -0.0834
Updated Alt_Multi_V2.xlsx with differences

All files processed success

In [ ]:
import zipfile
import os
from datetime import datetime

# Files to zip
files = ['Flux_Single.xlsx', 'Flux_Multi_V2.xlsx', 'Alt_Single.xlsx', 'Alt_Multi_V2.xlsx']

# Create zip filename with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'knowledge_scores_{timestamp}.zip'

try:
    print(f"Creating zip file: {zip_filename}")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in files:
            if os.path.exists(file):
                print(f"Adding {file} to zip...")
                zipf.write(file)
            else:
                print(f"Warning: {file} not found!")

    print(f"\nZip file created successfully: {zip_filename}")
    print(f"Location: {os.path.abspath(zip_filename)}")

except Exception as e:
    print(f"Error creating zip file: {e}")

Creating zip file: knowledge_scores_20250213_100555.zip
Adding Flux_Single.xlsx to zip...
Adding Flux_Multi_V2.xlsx to zip...
Adding Alt_Single.xlsx to zip...
Adding Alt_Multi_V2.xlsx to zip...

Zip file created successfully: knowledge_scores_20250213_100555.zip
Location: /content/knowledge_scores_20250213_100555.zip
